In [1]:
import numpy
from matplotlib import pyplot as plt
import matplotlib as mpl
from scipy import signal

In [6]:
seq1 = numpy.asarray([0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,  0, 0, 0, 1, 0, 0])
seq2 = numpy.asarray([22, 35, 32, 29])

sequences = [seq1, seq2]

# I - Resample sequences to match the one with the higher frequency (up-sampling using Fourier method)
result = []
m = max([len(s) for s in sequences])
for s in sequences:
    result.append(signal.resample(s, m))

# II - Compute covariance matrix between discrete signals
print(numpy.cov(result[0], result[1]))

# III - 

[array([  7.94685954e-16,   1.00000000e+00,   3.98803797e-16,
        -2.80477396e-16,   4.44089210e-16,  -3.94421338e-16,
         1.64780470e-15,   1.00000000e+00,   2.33731163e-17,
         3.09693791e-16,   3.91499698e-16,  -1.86984930e-16,
         1.00000000e+00,   1.67117782e-15,  -1.66533454e-16,
         3.03850512e-16,   1.00000000e+00,   8.16598251e-16,
        -4.67462326e-16]), array([ 22.        ,  23.77216093,  26.78322187,  30.28099721,
        33.37945774,  35.36855346,  35.94900103,  35.30002795,
        33.95784054,  32.56104718,  31.57347964,  31.10215618,
        30.88568449,  30.45436107,  29.38904651,  27.56305614,
        25.25799834,  23.0979456 ,  21.82396411])]
[[  0.1754386   -0.15467379]
 [ -0.15467379  21.24305556]]
